<a href="https://colab.research.google.com/github/soaresedirlan/data_science/blob/master/Previs%C3%A3o_de_Valores_Casas_Boston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Base de Dados Boston**

 ---

**`CRIM`**: Taxa de criminalidade per capita por regiao.

**`ZN`**: Proporção de terrenos residenciais divididos por lotes com mais de 25.000 pés quadrados.

**`INDUS`**: Essa é a proporção de hectares de negócios não comerciais por regiao.

**`CHAS`**: variável fictícia Charles River (= 1 se o trecho limita o rio; 0 caso contrário)

**`NOX`**: concentração de óxido nítrico (partes por 10 milhões)

**`RM`**: Número médio de quartos entre as casas do bairro

**`Age`**: proporção de unidades ocupadas pelos proprietários construídas antes de 1940

**`DIS`**: distâncias ponderadas para cinco centros de emprego em Boston

**`RAD`**: Índice de acessibilidade às rodovias radiais

**`IMPOSTO`**: taxa do imposto sobre a propriedade de valor total por US $ 10.000

**`B`**: 1000 (Bk - 0,63) ², onde Bk é a proporção de pessoas de descendência afro-americana por regiao

**`PTRATIO`**: Bairros com maior proporção de alunos para professores (maior valor de 'PTRATIO')

**`LSTAT`**: porcentagem de status mais baixo da população

**`MEDV`**: valor médio de casas ocupadas pelos proprietários em US $ 1000

Montar Google Drive



##Importar as bibliotecas Padrões


In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

##**Carregar base de dados**

In [0]:
from sklearn.datasets import load_boston

boston = load_boston()

In [0]:
#Descrição do dataset

print(boston.DESCR)

In [0]:
#Cria o dataframe do pandas
data = pd.DataFrame(boston.data, columns=boston.feature_names)

In [0]:
#Ver o dataframe
data.head()

In [0]:
data['MEDV'] = boston.target

In [0]:
#Escreva o arquivo no disco
data.to_csv('boston.csv')

## **Análise e Exploração dos Dados**

Nesta etapa nosso objetivo é conhecer os dados que estamos trabalhando.

Podemos a ferramenta **Pandas Profiling** para essa etapa:

In [0]:
# instalando o pandas profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [0]:
# executando o profile
from pandas_profiling import ProfileReport

profile = ProfileReport(data, title='Relatório - Pandas Profiling', html={'style':{'full_width':True}})

In [0]:
profile

**Observações**

*   *O coeficiente de correlação varia de `-1` a `1`. 
Se valor é próximo de 1, isto significa que existe uma forte correlação positiva entre as variáveis. Quando esse número é próximo de -1, as variáveis tem uma forte correlação negativa.*

*   *A relatório que executamos acima nos mostra que a nossa variável alvo (**MEDV**) é fortemente correlacionada com as variáveis `LSTAT` e `RM`*

*   *`RAD` e `TAX` são fortemente correlacionadas, podemos remove-las do nosso modelo para evitar a multi-colinearidade.*

*   *O mesmo acontece com as colunas `DIS` and `AGE` a qual tem a correlação de -0.75*

*   *A coluna `ZN` possui 73% de valores zero.*

In [0]:
#Salvando Report no disco
profile.to_file(output_file='Report_boston.html')

###Análise Manual

In [0]:
data.describe()

In [0]:
#Verificar valores missing
data.isnull().sum()

In [0]:
#calcule a correlação
correlacoes = data.corr()
#HeadMeap do seaborn
plt.figure(figsize=(16, 6))
sns.heatmap(data=correlacoes, annot=True)

Vizualizar relação entre algumas features e a variavel alvo.

In [0]:
#RM vs MEDV(Número de quartos e valor médio do Imovel)
import plotly.express as px

fig = px.scatter(data, x=data.RM, y=data.MEDV)
fig.show()

In [0]:
#PTRATIO vs MEDV(Percentual de proporção de alunos para provessores e valor médio do Imovel)
fig = px.scatter(data, x=data.PTRATIO, y=data.MEDV)
fig.show()

###Analisando Outliers

In [0]:
# estatística descritiva da variável RM
data.RM.describe()

In [0]:
# visualizando a distribuição da variável RM
import plotly.figure_factory as ff

labels = ['Distribuição da variável RM (número de quartos)']
fig = ff.create_distplot([data.RM], labels, bin_size=.2)
fig.show()

In [0]:
# Visualizando outliers na variável RM
import plotly.express as px

fig = px.box(data, y='RM')
fig.update_layout( width=800,height=800)
fig.show()

Visualizando a distribuição da variável MEDV

In [0]:
# estatística descritiva da variável MEDV
data.MEDV.describe()

In [0]:
# visualizando a distribuição da variável MEDV
import plotly.figure_factory as ff

labels = ['Distribuição da variável MEDV (preço médio do imóvel)']
fig = ff.create_distplot([data.MEDV], labels, bin_size=.2)
fig.show()

Analisando a simetria do dado

In [0]:
#Imprime o coeficiente de pearson
from scipy import stats

stats.skew(data.MEDV)

Coeficiente de Pearson
*   Valor entre -1 e 1 - distribuição simétrica.
*   Valor maior que 1 - distribuição assimétrica positiva.
*   Valor maior que -1 - distribuição assimétrica negativa.

In [0]:
# Histogram da variável MEDV (variável alvo)
fig = px.histogram(data, x="MEDV", nbins=50, opacity=0.50)
fig.show()

In [0]:
# Visualizando outliers na variável MEDV
import plotly.express as px

fig = px.box(data, y='MEDV')
fig.update_layout( width=800,height=800)
fig.show()

In [0]:
# estatistica descritiva das variáveis
data[['PTRATIO','LSTAT','RM']].describe()

In [0]:
# imprimindo os 16 maiores valores de MEDV
data[['RM','LSTAT','PTRATIO','MEDV']].nlargest(16, 'MEDV')

###Removendo Outliers

In [0]:
# filtra os top 16 maiores registro da coluna MEDV
top16 = data.nlargest(16, 'MEDV').index

In [0]:
# remove os valores listados em top16
data.drop(top16, inplace=True)

In [0]:
# visualizando a distribuição da variável MEDV
import plotly.figure_factory as ff

labels = ['Distribuição da variável MEDV (número de quartos)']
fig = ff.create_distplot([data.MEDV], labels, bin_size=.2)
fig.show()

In [0]:
# Histogram da variável MEDV (variável alvo)
fig = px.histogram(data, x="MEDV", nbins=50, opacity=0.50)
fig.show()

In [0]:
# imprime o coeficiente de pearson
stats.skew(data.MEDV)

## **Definindo um Baseline**

- Uma baseline é importante para ter marcos no projeto.
- Permite uma explicação fácil para todos os envolvidos.
- É algo que sempre tentaremos ganhar na medida do possível.

In [0]:
#converter Dados
data.RM =data.RM.astype(int)

In [0]:
data.info()

In [0]:
#Definindo a regra para categorizar os dados
categorias = []

In [0]:
#Alimenta a lista de Categorias
for i in data.RM.iteritems():
  valor = (i[1])
  if valor <= 4:
    categorias.append('Pequeno')
  elif valor < 7:
    categorias.append('Médio')
  else:
    categorias.append('Grande')

In [0]:
#Cria a categoria Categorias
data['categorias'] = categorias

In [0]:
# Imprime a contagem de categorias
data.categorias.value_counts()

In [0]:
#Agrupar as categorias e caucular as médias
medias_categorias = data.groupby(by='categorias')['MEDV'].mean()

In [0]:
#Vizualizando a variavel medias_categoricas
medias_categorias

In [0]:
#Criando o dicionario com chames médio, grande e pequeno e seus valores
dic_baseline = {'Grande': medias_categorias[0], 'Médio': medias_categorias[1], 'Pequeno': medias_categorias[2]}
dic_baseline

In [0]:
#Cria a função retorna baseline
def retorna_baseline(num_quartos):
  if num_quartos <= 4:
    return dic_baseline.get('Pequeno')
  elif num_quartos < 7:
    return dic_baseline.get('Médio')
  else:
    return dic_baseline.get('Grande')

In [0]:
#Chama funçaõ retorna baseline
retorna_baseline(1)

In [0]:
for i in data.RM.iteritems():
  n_quartos = i[1]
  print('Numero de Quartos é:', (n_quartos,' e o valor médio é:', retorna_baseline(n_quartos)))

In [0]:
data.head()

Preparando os dados

In [0]:
X = data.drop(['RAD', 'TAX', 'MEDV','DIS', 'AGE', 'ZN', 'categorias'], axis=1)
y = data['MEDV']

In [0]:
X.head()

##Separa conjunto de dados

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

##Machine Learning

###Modelo Baseline

In [0]:
#Definindo a lista de predições
predicoes = []

In [0]:
#Para cada elemento do teste alimenta a lista de predicoes
for i in X_test.RM.iteritems():
  n_quartos = i[1]
  predicoes.append(retorna_baseline(n_quartos))

In [0]:
predicoes[:10]

In [0]:
#Cria datafrae vazio
df_results = pd.DataFrame()

In [0]:
#Adiciona o valor real
df_results['valor_real'] = y_test.values

In [0]:
#Adiciona valor baseline
df_results['valor_baseline'] = predicoes

In [0]:
df_results.head()

Plotar o valor real e o baseline

In [0]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_baseline,
                         mode='lines+markers',
                         name='Valor Baseline'))
# Plota a figura
fig.show()

In [0]:
# calcula a métrica rmse
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = (np.sqrt(mean_squared_error(y_test, predicoes)))

In [0]:
# imprime a performance do modelo
print ('Performance do modelo baseline:')
print('\nRMSE é: {} '.format(rmse)) 

###Modelo Regressão Linear

- Devido a correlação entre algumas features podemos usar uma regressão linear.
- Modelo simples e eficiente.
- Solução rápida e robusta.


In [0]:
from sklearn.linear_model import LinearRegression

#Cria LinearRegression
lin_model = LinearRegression()

#Treina modelo LinearRegression
lin_model.fit(X_train, y_train)

#Avalia o Modelo nos dados de teste
y_pred = lin_model.predict(X_test)


In [0]:
df_results.head(10)

In [0]:
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
print ('Performance do modelo Regressão Linear:')
print('\nRMSE é: {} '.format(rmse)) 

In [0]:
# atribui os resultados no dataframe df_results
df_results['valor_reg_linear'] = lin_model.predict(X_test)

In [0]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_baseline,
                         mode='lines+markers',
                         name='Valor Baseline'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_reg_linear,
                         mode='lines',
                         line = dict(color = '#FEBFB3'),
                         name='Valor Regressão Linear'))

# Plota a figura
fig.show()

### Modelo Arvore de decisão

*   Algoritmo de aprendizado supervisionado
*   Explicabilidade do Modelo
*   Simples`

In [0]:
from sklearn.tree import DecisionTreeRegressor

#Criar Algoritimo DecisiontreeRegressor
regressor = DecisionTreeRegressor()

#Treina o modelo
regressor.fit(X_train, y_train)

#Fazendo Previsões
y_pred = regressor.predict(X_test)

In [0]:
#Adicionar o valor do modelo no dataframe df_results
df_results['Valor_Arvore_Decisao'] = y_pred

In [0]:
df_results.head(10)

In [0]:
import plotly.graph_objects as go

# cria uma figura
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_baseline,
                         mode='lines+markers',
                         name='Valor Baseline'))


# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_reg_linear,
                         mode='lines+markers',
                         name='Valor Regressão Linear'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.Valor_Arvore_Decisao,
                         mode='lines+markers',
                         name='Valor Arvore de Decisão'))
# Plota a figura
fig.show()

In [0]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

### Random Forest

- Método Ensemble
- Mais robustez para os modelos


Parâmetros da Random Forest

- O parâmetro n_estimators define a quantidade de árvores que serão usadas para criação dos modelos. Quanto maior o número de árvores, maior será o número de modelos no ensemble.

- O parâmtro max_deph define a profundidade da árvore, quanto mais profunda a árvore, mais especializado nos dados de treino o nosso modelo estará.

- O parâmetro max_features define a quantidade máxima de nós que serão usados para separar os níveis folha da árvore

In [0]:
from sklearn.ensemble import RandomForestRegressor

#Criar o algoritimo RandomForestRegressor
rf_regressor = RandomForestRegressor()

#Treina o modelo
rf_regressor.fit(X_train, y_train)

#Previsão do meu modelo
y_pred = rf_regressor.predict(X_test)

In [0]:
# adiciona os resultados no dataframe df_results
df_results['Valor_Random_forest'] = y_pred

In [0]:
#Calcula a metrica RMSE
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [0]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

In [0]:
# plota os resultados dos modelos e o valor real.
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_baseline,
                         mode='lines+markers',
                         name='Valor Baseline'))


# Linha com os dados preditos pela arvore
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.Valor_Arvore_Decisao,
                         mode='lines',
                         line = dict(color = '#B2FF66'),
                         name='Valor Árvore Decisão'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_reg_linear,
                         mode='lines',
                         line = dict(color = '#17BECF'),
                         name='Valor Regressão Linear'))


# Linha com os dados preditos pela Random Forest
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.Valor_Random_forest,
                         mode='lines',
                         line = dict(color = '#7F7F7F'),
                         name='Valor Random Forest'))

# Plota a figura
fig.show()